<h1 style='text-align:center; font-size:30px; font-weight:bold; '>Fine-Tuning Distilbert on the FinancialPhraseBank Dataset</h1>

# Introduction

## Table of Contents

1. Introduction and Objective
2. Dataset Loading and Preprocessing
3. Exploratory Data Analysis
4. Baseline Classifier Head Only (Frozen Encoder)
5. Fine-Tune All Weights (Using Trained Classifier Head)
6. Fine-Tune All Weights from Scratch
7. Evaluation & Comparison
8. Discussion & Reflection


In [4]:
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 0


Installing libraries & adding imports

In [2]:
# !pip install transformers datasets scikit-learn pandas numpy tqdm tensorflow

import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random
import os

import warnings
warnings.filterwarnings("ignore")

2025-04-25 12:22:42.168758: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Financial PhraseBank Dataset

### Alternative Model Considerations

| Model | Parameters | Notes |
|-------|------------|-------|
| `distilbert-base-uncased` | ~66M | Lightweight, fast to train. Chosen as baseline and required for Part 2. |
| `bert-base-uncased` | ~110M | More expressive, but slower. Considered for extra experiments. |
| `albert-base-v2` | ~12M | Extremely compact due to weight sharing. Good for parameter-efficiency testing. |
| `roberta-base` | ~125M | High-performing, but uses a different tokenizer. Reserved for advanced exploration. |
| `electra-small-discriminator` | ~14M | Fast and efficient, but less common in TensorFlow workflows. Not chosen for core tasks. |

**`distilbert-base-uncased`** is selected due to:
- Smaller size compared to `bert-base-uncased`, allowing for a broader range of ablation experiments
- Solid performance on general sentiment tasks
- Compatibility with Part 2 requirements, which involve applying LoRA adapters to the FFN (`lin1`, `lin2`) layers of `distilbert-base-uncased` to evaluate parameter-efficient fine-tuning

Other models may be explored in separate sections to assess the impact of architecture and scale on downstream performance.



In [3]:
# Load the "all agree" subset
dataset = load_dataset("financial_phrasebank", "sentences_allagree") # All agree signifies 100% of annotators agreed on sentiment of this subset

# Peek at the data
dataset["train"][0]

{'sentence': 'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .',
 'label': 1}

# Fine-Tuning: Classifier Head Only


- 3 attempts

In [ ]:
# tokenizer
def tokenize_function

## Results

In [ ]:
# Save weights

# Fine-Tune All Layers Using Previous Classifier Head

In [ ]:
#

# Entire Model: Pre-Trained + Classifier Head